In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tvb.simulator.lab as tsl
import mne_connectivity 
import tvb
from scipy import signal
from importlib import reload
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn.cluster import KMeans
import scipy 
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd
import netplotbrain

import functions as fn

In [ ]:
def DFC_visu_pipeline(data, cor_method, num_clusters=3, window_size=2000, overlap=1000,tmax=30, intrahemispheric=False, square=[0,0],animated_visu=True,ts=0.5):
    '''
    This function performs visualisations regarding Dynamical Functional connectivity (DFC). 

    ------------
    INPUTS
    ------------

    data : array_like, shape = (signal_duration/timestep,num_regions) 
        The signal data, typically eeg signal or source level signal. 
    cor_method : string
        The correlation method to build the connectivity matrices. Can be chosen over 'Pearson', 'Spearman', 'Coherence', 'Phase-lock' and 'wpli'.
    num_clusters : int, default = 3
        Number of clusters for the k-means clustering.
    window_size : int, default = 2000  
        window_duration[ms]/timestep[ms]
    overlap : int or float, default = 1000 
        Time of overlap [ms] between two succint windows. 
    tmax : int, default = 30
        Time of the simulation, for visualisation purpose. 
    intrahemispheric : bool, default = False
        Visualisation of clustering of a single square of the connectivity matrix. 
    square : 2x2 list, default = [0,0]
        If intrahemispheric = True, specifies the square of the matrix to perform the analysis on. 
    animated_visu : bool, default = True
        Plot and save an animated plot showing the evolution of correlation matrices and pca visualisation. 


    ------------
    OUTPUTS
    ------------
    closest_mats : array_like, shape = (num_clusters, num_regions,num_regions)
        Closest connectivity matrix to every cluster. 
    most_repeating_states : array_like, shape = (num_clusters, num_regions,num_regions)
         Kmeans clusters centroids. 
    correlation_matrices : array_like, shape = (num_windows, num_regions,num_regions)
        Set of correlation matrics, one per window. 
    cluster_labels : list, len = num_clusters
        Cluster labels.
    distances : array_like, shape = (num_clusters, num_windows)
         Matrix storing the distances between each connectivity matrix and each cluster. 
    pca : Sklearn PCA object.
        PCA object used for the connectivity matrices dimension reduction.   
    reduced_data : array_like
        3D PCA reduced data.
    '''
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,num_clusters = fn.build_FCmat(data,cor_method,num_clusters,window_size,overlap,tmax,intrahemispheric,square=square)
    distances = fn.compute_clst_dist(most_repeating_states, correlation_matrices, cluster_labels,tmax,num_clusters) 
    pca, reduced_data = fn.plot_PCA(correlation_matrices, most_repeating_states, cluster_labels,num_clusters)
    if animated_visu : 
        fn.animated_plot(correlation_matrices,reduced_data,cluster_labels)

    return closest_mats,most_repeating_states, correlation_matrices, cluster_labels, distances, pca, reduced_data
    


# Define paths

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'

# Effects of window len

In [ ]:
Wins=[0.5,1,1.5,2,3,4,5,8,10] #s
timestep = 0.5 #ms
Wins=[i/timestep for i in Wins]
# Number of clusters (most repeating states)
num_clusters = 3
mats=np.zeros((len(Wins), num_clusters,76,76))
for i,w in enumerate(Wins):
    data = np.load(f'{path_save}/test1.5minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 80000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels, distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,window_size=w*1000,overlap=w*500,tmax=t2/1000,animated_visu=False)
        mats[i]=closest_mats
    

# 40 Minutes stim

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 1200000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

In [ ]:
custom_colors = ['red', 'blue', 'green']
colors=[]
sizes=[]
for i in range(len(correlation_matrices)):
    if i<21:
        colors.append([custom_colors[cluster_labels[j]] for j in range(i+1)])
        sizes.append((i+1)*[10])
    else : 
        colors.append(['black'] * (i-20) + [custom_colors[cluster_labels[j]] for j in range(i-20,i)])
        sizes.append((i-20)*[4]+20*[10])


# Create the figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=['Matrix Evolution', 'PCA Evolution'],specs=[
        [{"type": "Heatmap"}, {"type": "scatter3d"}]])

# Initialize the first subplot with the initial matrix heatmap
heatmap_trace = go.Heatmap(z=np.flip(correlation_matrices[0], axis=0), colorscale='Viridis')
fig.add_trace(heatmap_trace, row=1, col=1)

# Initialize the second subplot with the initial PCA scatter plot
scatter_trace = go.Scatter3d(x=reduced_data[:, 0], y=reduced_data[:, 1], z=reduced_data[:, 2], mode='markers',marker=dict(color=cluster_labels,size=2,colorscale=custom_colors ))
fig.add_trace(scatter_trace, row=1, col=2)
fig.frames=[go.Frame(data=[go.Heatmap(z=np.flip(correlation_matrices[i], axis=0), colorscale='Viridis'),go.Scatter3d(x=reduced_data[:i, 0], y=reduced_data[:i, 1], z=reduced_data[:i, 2], mode='markers',marker=dict(color=cluster_labels,size=sizes[i],colorscale=custom_colors ))], traces=[0,1]) for i in range(1,len(correlation_matrices))]

# Update layout to include animation settings
fig.update_layout(updatemenus=[dict(type='buttons',
                                showactive=False,
                                buttons=[dict(label='Play',
                                                method='animate',
                                                args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True)])
                                                , 
                                                {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                                                        "mode": "immediate",
                                                        "transition": {"duration": 0}}],
                                                        "label": "Pause",
                                                        "method": "animate"
                                                }])],height=1000, width=2000)

fig.show()
pio.write_html(fig,auto_play=False,file='cloud_template_conn.html')

## Animated plot

In [ ]:
custom_colors = ['red', 'blue', 'green']

# Create the figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=['Matrix Evolution', 'PCA Evolution'],specs=[
        [{"type": "Heatmap"}, {"type": "scatter3d"}]])

# Initialize the first subplot with the initial matrix heatmap
heatmap_trace = go.Heatmap(z=np.flip(correlation_matrices[0], axis=0), colorscale='Viridis')
#heatmap_frame = [go.Frame(data=[go.Heatmap(z=correlation_matrices[i], colorscale='Viridis')], name=f'Frame{i}') for i in range(1, len(correlation_matrices))]
fig.add_trace(heatmap_trace, row=1, col=1)

# Initialize the second subplot with the initial PCA scatter plot
scatter_trace = go.Scatter3d(x=reduced_data[:, 0], y=reduced_data[:, 1], z=reduced_data[:, 2], mode='markers',marker=dict(color=cluster_labels,size=2,colorscale=custom_colors ))
#scatter_frame = [go.Frame(data=[go.Scatter3d(x=reduced_data[:i, 0], y=reduced_data[:i, 1], z=reduced_data[:i, 2], mode='markers')], name=f'Frame{i}') for i in range(1, len(correlation_matrices))]
fig.add_trace(scatter_trace, row=1, col=2)
fig.frames=[go.Frame(data=[go.Heatmap(z=np.flip(correlation_matrices[i], axis=0), colorscale='Viridis'),go.Scatter3d(x=reduced_data[:i, 0], y=reduced_data[:i, 1], z=reduced_data[:i, 2], mode='markers',marker=dict(color=cluster_labels,size=2,colorscale=custom_colors ))], traces=[0,1]) for i in range(1,len(correlation_matrices))]
# Update layout to include animation settings

fig.update_layout(updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True)])
                                                  , 
                                                  {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                                                        "mode": "immediate",
                                                        "transition": {"duration": 0}}],
                                                        "label": "Pause",
                                                        "method": "animate"
                                                }])],height=1000, width=2000)

# Show the plot
fig.show()
pio.write_html(fig,auto_play=False,file='animated_plot.html')

## Adding clusters

In [ ]:
closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000, num_clusters=3)

## Flipping hemispheres

In [ ]:
def square_flip(mat):
    squares=np.zeros((2,2,int(len(mat)/2),int(len(mat)/2)))
    for i in range(2) : 
        for j in range(2):
            squares[i,j]=mat[i*int(len(mat)/2):(i+1)*int(len(mat)/2),j*int(len(mat)/2):(j+1)*int(len(mat)/2)]
    mat=np.hstack([np.concatenate((squares[1,1],squares[0,1])),np.concatenate((squares[1,0],squares[0,0]))])
    return mat

def inter_flip(mat):
    squares=np.zeros((2,2,int(len(mat)/2),int(len(mat)/2)))
    for i in range(2) : 
        for j in range(2):
            squares[i,j]=mat[i*int(len(mat)/2):(i+1)*int(len(mat)/2),j*int(len(mat)/2):(j+1)*int(len(mat)/2)]
    mat=np.hstack([np.concatenate((squares[0,0],squares[0,1])),np.concatenate((squares[1,0],squares[1,1]))])
    return mat

def intra_flip(mat):
    squares=np.zeros((2,2,int(len(mat)/2),int(len(mat)/2)))
    for i in range(2) : 
        for j in range(2):
            squares[i,j]=mat[i*int(len(mat)/2):(i+1)*int(len(mat)/2),j*int(len(mat)/2):(j+1)*int(len(mat)/2)]
    mat=np.hstack([np.concatenate((squares[1,1],squares[1,0])),np.concatenate((squares[0,1],squares[0,0]))])
    return mat

In [ ]:
# 3D PCA
num_clusters=3
np.random.seed=42
rmd_idx=np.random.randint(1,957,10)
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,5776)
#flipped_mtc=[inter_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
flipped_mtc=[intra_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
pca_ref=PCA(n_components=3)
reduced_data = pca_ref.fit_transform(data)
flip_rshp = np.array(flipped_mtc).reshape(len(flipped_mtc),5776)
flip_pca=pca_ref.transform(flip_rshp)
centroids = reduced_data[-num_clusters:]

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-3, 0], y=reduced_data[:-3, 1], z=reduced_data[:-3, 2],
                                mode='markers', marker=dict(color='black'), showlegend=False)])

fig.add_trace(go.Scatter3d(x=reduced_data[rmd_idx, 0], y=reduced_data[rmd_idx, 1], z=reduced_data[rmd_idx, 2],
                                mode='markers', marker=dict(color='red'), name='reference points'))

fig.add_trace(go.Scatter3d(x=flip_pca[:, 0], y=flip_pca[:, 1], z=flip_pca[:, 2],
                                mode='markers', marker=dict(color='orange'), name='intrahemispheric flipped references'))

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))
for i in range(len(rmd_idx)):
    fig.add_trace(go.Scatter3d(
        x=[reduced_data[rmd_idx[i], 0], flip_pca[i, 0]],
        y=[reduced_data[rmd_idx[i], 1], flip_pca[i, 1]],
        z=[reduced_data[rmd_idx[i], 2], flip_pca[i, 2]],
        mode='lines',
        line=dict(color='blue'),
        showlegend=False
    ))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"Arrows indicate temporal order", 
    height=1000, width=1000)
fig.show()



In [ ]:
plt.imshow(correlation_matrices[0])

In [ ]:
plt.imshow(square_flip(correlation_matrices[0]))

In [ ]:
components = pca.components_
for comp in components : 
    plt.figure()
    plt.imshow(comp.reshape(76,76),cmap='bwr')
    plt.colorbar();plt.clim([-0.05,0.05])

## distance to cluster + z-axis coordinates

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((num_clusters,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((num_clusters)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-most_repeating_states[j]))

sums=np.sum(distances,axis=0)

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]

colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-num_clusters, 2]/np.max(reduced_data[:, 2]), 40, 4),label='Smoothed relative PCA z-coordinate')
for p in range((num_clusters)):
    plt.plot(time,distances[p,:],label=str(p),c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()

plt.show()

plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,num_clusters,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()

## 5 min sim

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 600000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000,animated_visu=False)

# Manual cluster creation

In [ ]:
for mat in most_repeating_states : 
    plt.figure()
    plt.imshow(mat)

In [ ]:
# 3D PCA
num_clusters=3
manual_clusters = [most_repeating_states[0],most_repeating_states[1],pca.inverse_transform([10,-5.5,0]).reshape(76,76),pca.inverse_transform([10,7,0]).reshape(76,76)]
ext_data=np.concatenate((correlation_matrices,manual_clusters))
data=np.array(ext_data).reshape(len(correlation_matrices)+4,5776)
pca=PCA(n_components=3)
reduced_data = pca.fit_transform(data)
manual_clusters = [most_repeating_states[0],most_repeating_states[1],pca.inverse_transform([10,-5.5,0]).reshape(76,76),pca.inverse_transform([10,7,0]).reshape(76,76)]

centroids = reduced_data[-4:,:]
labels=['right','left','top','bottom']

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-4, 0], y=reduced_data[:-4, 1], z=reduced_data[:-4, 2],
                                mode='markers', marker=dict(color='black', size=3))])


fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=7),
                            text=[labels[i] for i in range(len(labels))], name='Centroids', showlegend=False))




fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Manually added Centroids are marked with cross\n", 
    height=1000, width=1000)
fig.show()



In [ ]:
plt.imshow(pca.inverse_transform([10,-5.5,0]).reshape(76,76))

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((4,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((4)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-manual_clusters[j]))

sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((4)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((4)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((4,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((4)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-manual_clusters[j]))

sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((4)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((4)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
gc_res=[]
for p in range((4)):
    print(labels[p])
    df=pd.DataFrame(columns=['distances','z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1])
    gc_res.append(grangercausalitytests(df.values, 4,verbose=1))
    


In [ ]:
print(gc_res[0])

In [ ]:
df.values

In [ ]:
for i in manual_clusters : 
    plt.figure()
    plt.imshow(i)

# Bigger levels of noise

In [ ]:
sigma = [1e-3,1e-5,1e-6] 
for s in sigma : 
    data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

In [ ]:
sigma = np.linspace(1e-6,1e-7,20)#[1e-5,1e-6,9e-7,5e-7,3e-7,2e-7,1.5e-7,1e-7] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    build_FCmat(data_corr,'Pearson',num_clusters=3,window_size=2000,overlap=1000,tmax=30)

## smaller noise

In [ ]:
sigma = np.linspace(1e-12,0,20) #sigma = np.linspace(1e-7,1e-12,20)#[1e-5,1e-6,9e-7,5e-7,3e-7,2e-7,1.5e-7,1e-7] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{np.round(s,3)}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    build_FCmat(data_corr,'Pearson',num_clusters=3,window_size=2000,overlap=1000,tmax=30)

In [ ]:
sigma = [1e-10,1e-12,0] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth,window_size=2000,overlap=1000)

# Effect of G

# Long term intrahemispheric

In [ ]:
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=True, square=[1,0])

## z to intrahemispheric

In [ ]:
from scipy.signal import savgol_filter
sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((3)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((3)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

# Directed connectivity

In [ ]:
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 30000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['wpli']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth, window_size=2000,overlap=1000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
from PyIF import te_compute as te
data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{1e-7}.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

conmat=np.zeros((len(data_corr.T),len(data_corr.T)))
for i in range(len(data_corr.T)):
    for j in range(len(data_corr.T)):
        conmat[i,j] = te.te_compute(data_corr[:,i], data_corr[:,j], k=1, embedding=1, safetyCheck=True, GPU=False)

plt.imshow(conmat)

# New subject TVB2

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB2.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca2, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
def pca_explained_variance(pca):
    return np.cumsum(pca.explained_variance_ratio_)

In [ ]:
print(pca_explained_variance(pca))

In [ ]:
pca.explained_variance_ratio_

# TVB10

In [ ]:

G=[7]
sigma=[1e-9]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for s in range(len(sigma)):
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 300000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## 40 minutes

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'

G=[5,7]
sigma = [1e-7,1e-9,1e-11]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for g in range(len(G)):
    for s in range(len(sigma)):
        print(G[g],' ',sigma[s])
        data = np.load(f'{path_save}/40sim_dt_0.5_G_{G[g]}_sigma_{sigma[s]}_TVB10.npz')
        data_time=data['traw_time']
        delta_t = data_time[1]-data_time[0]
        sf = 1/(delta_t*1e-3)
        win = 1*sf

        t1 = 1000
        t2 = 2400000 #unit second
        time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
        idc1 = time_corr_id[0][0]
        idc2= time_corr_id[0][-1]
        time_corr=data_time[idc1:idc2]
        data_corr=data['traw_y1y2'][idc1:idc2]
        print(np.array(data_corr).shape)

        methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
        for meth in methods : 
            closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca10, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=True, square=[1,1])

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

### Intrahemispheric

In [ ]:
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-09}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth,intrahemispheric=False,square=[0,1], window_size=100, overlap=50)

### PCA ref transform

In [ ]:
# 3D PCA
num_clusters=3
np.random.seed=42
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,len(correlation_matrices[0])**2)
#flipped_mtc=[inter_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
pre_reduced= PCA(n_components=76**2).fit_transform(data)
reduced_data = pca_ref.transform(pre_reduced)
centroids = reduced_data[-num_clusters:]

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], z=reduced_data[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = reduced_data[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=reduced_data[:-num_clusters, 0],y=reduced_data[:-num_clusters, 1],z=reduced_data[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"Arrows indicate temporal order", 
    height=1000, width=1000)
fig.show()



# TVB5

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
plot_PCs(pca)

## 40 min

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca5, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

## Intrahemispheric

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca5, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=True, square=[1,1])

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

# TVB7

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB7.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

## Try normalisation 

In [ ]:
from sklearn import preprocessing

path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB7.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
data_corr = preprocessing.normalize(data_corr, axis=0)
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca7, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Try Granger

In [ ]:
import numpy as np
from statsmodels.tsa.api import VAR

# Initialize the directed functional connectivity matrix
connectivity_matrix = np.zeros((84,84))
data = data_corr
windowed_data = data[1000:1000 + int(2000), :]

# Fit a VAR model to the data
model = VAR(windowed_data)

# Compute the lag order using an information criterion (e.g., AIC or BIC)
lag_order = 4

# Estimate the coefficients of the VAR model
model_fit = model.fit(lag_order)

# Compute the Granger causality matrix
granger_matrix = model_fit.test_causality(caused=range(84), causing=range(84), kind='f')

# Update the directed functional connectivity matrix with the Granger causality values
connectivity_matrix = granger_matrix.reshape(84, 84)

# Print the directed functional connectivity matrix
print(connectivity_matrix)


In [ ]:
len(windowed_data.T)

In [ ]:
granger_matrix.summary()

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

tst=np.zeros(84*84)

for i in range(84):
    for j in range(84):
        df=pd.DataFrame(columns=['1','2'])
        df['1']=windowed_data[:,i]
        df['2']=windowed_data[:,j]
        x=grangercausalitytests(df.values, [2],verbose=1)
        tst[i*84+j]=x[2][0]['params_ftest'][1]
    

In [ ]:
plt.imshow(tst.reshape(84,84))
plt.colorbar()

In [ ]:
print(x)

# TVB8

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB8.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB9

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB9.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca2, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB11

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB11.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca11, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB12

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB12.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca12, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# PCs comparison

In [ ]:
pcas=[pca2,pca5,pca7,pca8,pca9,pca10,pca11,pca12]
str_pcas=['pca2','pca5','pca7','pca8','pca9','pca10','pca11','pca12']
comps=[]
for p,pca in enumerate(pcas) : 
    comps.append(plot_PCs(pca))

In [ ]:
sp_comps = []
for c in comps : 
    for cc in c : 
        sp_comps.append(cc)
        
 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-0.05, 0.05))
sp_comps = scaler.fit_transform(sp_comps)
for i, c in enumerate(sp_comps): 
    sp_comps = scaler.fit_transform(sp_comps)
    for j, com in enumerate(sp_comps): 
        if i != j: 
            diff = c - com
            plt.figure()
            plt.imshow(diff.reshape(84, 84),cmap='bwr')
            plt.colorbar();plt.clim([-0.05,0.05])
            plt.title(f'PC{np.floor(i/3)+1}, {str_pcas[i%3]} - PC{np.floor(j/3)+1}, {str_pcas[j%3]}')


# Angle evolution

In [ ]:
import numpy as np

def calculate_angle(point1, point2):
    vector1 = np.array(point1)
    vector2 = np.array(point2)
    
    dot_product = np.dot(vector1, vector2)
    magnitude_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
    
    angle = np.arccos(dot_product / magnitude_product)
    
    return np.degrees(angle)



In [ ]:
plt.plot([calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)])

In [ ]:

def calculate_phase(point1, point2):
    phase1 = np.angle(point1,deg=True)
    phase2 = np.angle(point2,deg=True)
    
    phase_diff = phase2 - phase1
    
    return phase_diff


angles = [calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]
phases = [calculate_phase(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]

plt.scatter(angles, phases, c=phases, cmap='coolwarm')
plt.colorbar(label='Phase Difference')
plt.xlabel('Angle')
plt.ylabel('Phase Difference')
plt.title('Angle vs Phase Difference')
plt.show()


In [ ]:
angles = [calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]


fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], z=reduced_data[:-num_clusters, 2],
                                mode='markers', marker=dict(color='black',size=2))])

centroids = reduced_data[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=reduced_data[:-num_clusters, 0],y=reduced_data[:-num_clusters, 1],z=reduced_data[:-num_clusters, 2],mode='lines',line=dict(color=angles*100, width=2)))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

In [ ]:
angles

# EEG study Fariba

## Before stim

### Stroke

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [5]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats5,most_repeating_states5, correlation_matrices5, cluster_labels5, distances5, pca5, reduced_data5 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
fig, axs = plt.subplots(1, num_clusters, figsize=(15, 5))
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Position of the colorbar

for a in range(len(closest_mats5)):
    im = axs[a].imshow(closest_mats5[a], cmap='viridis')
    axs[a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[a].axhline(y=34, color='white', linewidth=2)
    axs[a].axhline(y=49, color='white', linewidth=2)
    axs[a].axvline(x=34, color='white', linewidth=2)
    axs[a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')

fig.colorbar(im, cax=cbar_ax)
fig.suptitle(f'Closest matrix to the cluster centers, Stroke', y=0.92)
plt.show()


### Healthy

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats7,most_repeating_states7, correlation_matrices7, cluster_labels7,distances7, pca7, reduced_data7 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [10]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats10,most_repeating_states10, correlation_matrices10, cluster_labels10,distances10, pca10, reduced_data10 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## matrices figure

In [ ]:
fig, axs = plt.subplots(2, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats5)):
    im = axs[0, a].imshow(closest_mats5[a], cmap='viridis')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)

    
    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[0, 1].set_title('Stroke',fontsize=22)  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats7)):
    im = axs[1, a].imshow(closest_mats7[a], cmap='viridis')
    axs[1, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[1, a].axhline(y=34, color='white', linewidth=2)
    axs[1, a].axhline(y=49, color='white', linewidth=2)
    axs[1, a].axvline(x=34, color='white', linewidth=2)
    axs[1, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[1, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[1, 1].set_title('Healthy', fontsize=22)  # Add title for axs[1]

# Place colorbar at the bottom horizontally
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])

fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
plt.show()




## 2d PCA

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(15,5))

ext_data=np.concatenate((correlation_matrices5,most_repeating_states5))
data=np.array(ext_data).reshape(len(correlation_matrices5)+num_clusters,len(correlation_matrices5[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[0].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels5, palette='Set1', ax=axs[0])
#axs[0].legend(loc='upper left')

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[0]
)

axs[0].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[0].set_title(
    "Stroke", fontsize=22
)
axs[0].set_xlim(x_min-7, x_max)
axs[0].set_xticks(())
axs[0].set_yticks(())

ext_data=np.concatenate((correlation_matrices7,most_repeating_states7))
data=np.array(ext_data).reshape(len(correlation_matrices7)+num_clusters,len(correlation_matrices7[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[1].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels7, palette='Set1', ax=axs[1])

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[1]
)

axs[1].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[1].set_title(
    "Healthy", fontsize=22
)
axs[1].set_xlim(x_min-7, x_max)
axs[1].set_xticks(())
axs[1].set_yticks(())
axs[1].get_legend().remove()

## Proximity plots

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20, 15))
time = np.linspace(1,30,len(correlation_matrices5))

colors=['red','blue','green','orange','black','purple']


for a in range(num_clusters):    
    axs[0].plot(time, distances5[a, :], label=f'State {str(a)}', c=colors[a])
    axs[1].plot(time, distances7[a, :], label=f'State {str(a)}', c=colors[a])


axs[0].set_title('Stroke', fontsize=30)
axs[0].set_ylabel('1-distance',  fontsize=24)
axs[0].legend(fontsize=20)
axs[0].tick_params(axis='y', labelsize=18)
axs[0].tick_params(axis='x', labelsize=0)
axs[0].set_ylim(0.6,0.75)

axs[1].set_title('Healthy', fontsize=30)
axs[1].set_xlabel('Time [s]', fontsize=22)
axs[1].set_ylabel('1-distance',  fontsize=22)
axs[1].set_ylim(0.6,0.75)

axs[1].tick_params(axis='x', labelsize=18)
axs[1].tick_params(axis='y', labelsize=18)

plt.show()


In [ ]:
fig, axs = plt.subplots(4, num_clusters, figsize=(25, 15))
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Position of the colorbar
time = np.linspace(1,30,len(correlation_matrices))
colors=['red','blue','green','orange','black','purple']

# Plot closest_mats5
for a in range(len(closest_mats5)):
    im = axs[0, a].imshow(closest_mats5[a], cmap='plasma')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')
axs[0, 1].set_title('Stroke')  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats7)):
    im = axs[2, a].imshow(closest_mats7[a], cmap='plasma')
    axs[2, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[2, a].axhline(y=34, color='white', linewidth=2)
    axs[2, a].axhline(y=49, color='white', linewidth=2)
    axs[2, a].axvline(x=34, color='white', linewidth=2)
    axs[2, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[2, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[2, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[2, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')
axs[2, 1].set_title('Healthy')  # Add title for axs[1]

for a in range(num_clusters):
    axs[1, 1].plot(time, distances5[a, :], label=str(a), c=colors[a])
    axs[3, 1].plot(time, distances7[a, :], label=str(a), c=colors[a])

# Set the limits of the y-axis for the distance plots
axs[1, 1].set_ylim([0, 1])
axs[3, 1].set_ylim([0, 1])

# Remove unnecessary subplots
axs[1, 0].remove()
axs[3, 0].remove()
axs[1, 2].remove()
axs[3, 2].remove()

# Set common x and y labels for the remaining subplots
axs[1, 1].set_xlabel('Time [s]')
axs[1, 1].set_ylabel('1 - Relative distance')
axs[3, 1].set_xlabel('Time [s]')
axs[3, 1].set_ylabel('1 - Relative distance')

# Add legends to the remaining subplots
axs[1, 1].legend()
axs[3, 1].legend()

fig.colorbar(im, cax=cbar_ax)
fig.suptitle(f'Comparison of network states without stimulation between stroke and healthy patient', y=0.92)
plt.show()


## During stim

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [5,10]:#[2,5,7,8,9,10,11,12]    
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=30000
    t2=60000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca12, reduced_data = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Figure poster

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [7]:#[2,5,7,8,9,10,11,12]    
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=30000
    t2=60000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats_stim,most_repeating_states_stim, correlation_matrices_stim, cluster_labels_stim,distances_stim, pca12_stim, reduced_data_stim = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
fig, axs = plt.subplots(2, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats7)):
    im = axs[0, a].imshow(closest_mats7[a], cmap='viridis')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)
    
    # Outline the 13th row and 55th column
    axs[0, a].axhline(y=12, color='red', linewidth=0.5)
    axs[0, a].axvline(x=54, color='red', linewidth=0.5)
    axs[0, a].axhline(y=14, color='red', linewidth=0.5)
    axs[0, a].axvline(x=56, color='red', linewidth=0.5)

    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[0, 1].set_title('Before stimulation',fontsize=14)  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats_stim)):
    im = axs[1, a].imshow(closest_mats_stim[a], cmap='viridis')
    axs[1, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[1, a].axhline(y=34, color='white', linewidth=2)
    axs[1, a].axhline(y=49, color='white', linewidth=2)
    axs[1, a].axvline(x=34, color='white', linewidth=2)
    axs[1, a].axvline(x=49, color='white', linewidth=2)
    
    # Outline the 13th row and 54th column
    axs[1, a].axhline(y=12, color='red', linewidth=0.5)
    axs[1, a].axvline(x=54, color='red', linewidth=0.5)
    axs[1, a].axhline(y=14, color='red', linewidth=0.5)
    axs[1, a].axvline(x=56, color='red', linewidth=0.5)

    if a == 0:
        axs[1, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[1, 1].set_title('During stimulation', fontsize=14)  # Add title for axs[1]

# Place colorbar at the bottom horizontally
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])

fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
plt.show()


In [ ]:
fig,axs=plt.subplots(1,2,figsize=(15,5))

ext_data=np.concatenate((correlation_matrices_stim,most_repeating_states_stim))
data=np.array(ext_data).reshape(len(correlation_matrices_stim)+num_clusters,len(correlation_matrices_stim[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[0].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels_stim, palette='Set1', ax=axs[0])
#axs[0].legend(loc='upper left')

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[0]
)

axs[0].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[0].set_title(
    "Stroke", fontsize=22
)
axs[0].set_xlim(x_min-7, x_max)
axs[0].set_xticks(())
axs[0].set_yticks(())

ext_data=np.concatenate((correlation_matrices7,most_repeating_states7))
data=np.array(ext_data).reshape(len(correlation_matrices7)+num_clusters,len(correlation_matrices7[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[1].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels7, palette='Set1', ax=axs[1])

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[1]
)

axs[1].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[1].set_title(
    "Healthy", fontsize=22
)
axs[1].set_xlim(x_min-7, x_max)
axs[1].set_xticks(())
axs[1].set_yticks(())
axs[1].get_legend().remove()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(25, 20))
time = np.linspace(1,30,len(correlation_matrices))

colors=['red','blue','green','orange','black','purple']


for a in range(num_clusters):    
    axs[0].plot(time, distances7[a, :], label=f'State {str(a)}', c=colors[a])
    axs[1].plot(time, distances_stim[a, 4:], label=f'State {str(a)}', c=colors[a])


axs[0].set_title('Before stimulation', fontsize=30)
axs[0].set_xlabel('Time [s]', fontsize=24)
axs[0].set_ylabel('1-Distance',  fontsize=24)
axs[0].legend()

axs[1].set_title('After stimulation', fontsize=30)
axs[1].set_xlabel('Time [s]', fontsize=24)
axs[1].set_ylabel('1-Distance',  fontsize=24)
axs[1].legend()

fig.suptitle(f'Proximity to each state over time', y=0.94,fontsize=35)

plt.show()




## Results template

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
G=[7]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for g in range(len(G)):
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Timing_optimisation'
    data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats_temp,most_repeating_states_temp, correlation_matrices_temp, cluster_labels_temp,distances_temp, pca12_temp, reduced_data_temp = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
components = pca12_temp.components_
    for comp in components : 
        plt.figure()
        plt.imshow(comp.reshape(int(np.sqrt(len(comp))),int(np.sqrt(len(comp)))),cmap='bwr')
        plt.colorbar();plt.clim([-0.05,0.05])

In [ ]:
plot_PCs(pca12_temp)

In [ ]:
fig, axs = plt.subplots(1, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats_temp)):
    im = axs[ a].imshow(closest_mats_temp[a], cmap='viridis')
    axs[ a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[ a].axhline(y=37, color='white', linewidth=2)
    axs[ a].axvline(x=37, color='white', linewidth=2)

    
    if a == 0:
        axs[ a].text(-2.8, 33, 'Right hemisphere', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[ a].text(-2.8, 70, 'Left hemisphere', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[ a].text(18, 80, 'Right hemisphere', color='black', ha='center', rotation='horizontal',fontsize=11)
        axs[ a].text(55, 80, 'Left hemisphere', color='black', ha='center', rotation='horizontal',fontsize=11)

# Place colorbar on the right side vertically
cbar_ax = fig.add_axes([0.92, 0.325, 0.02, 0.34])

fig.colorbar(im, cax=cbar_ax, orientation='vertical')
plt.show()



In [ ]:
ext_data=np.concatenate((correlation_matrices_temp,most_repeating_states_temp))
data=np.array(ext_data).reshape(len(correlation_matrices_temp)+num_clusters,len(correlation_matrices_temp[0])**2)
pca_2d_temp=PCA(n_components=2)
reduced_data = pca_2d_temp.fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


plt.plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1],linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10)#,hue=cluster_labels,palette='Set1')



plt.quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid",width=0.001,headwidth=20)

# Plot the centroids as a white X
'''centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0,num_clusters-1,num_clusters),
    palette='Set1',
    legend=False
)'''


plt.xlim(x_min-7, x_max)
plt.xticks(())
plt.yticks(())
plt.legend('',frameon=False)
plt.show()

In [ ]:
coords[:, 1]

In [ ]:
centroids

In [ ]:
coords=np.array([[-13.5,0.4],
        [35,0.4],
        [7,-6],
        [7,7]])
for coord in coords : 
    plt.figure()
    plt.imshow(pca_2d_temp.inverse_transform(coord).reshape(76,76),cmap='viridis')
    plt.axis('off')


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 7.5))

components = pca_2d_temp.components_
for c, comp in enumerate(components):
    im=axs[c].imshow(comp.reshape(int(np.sqrt(len(comp))), int(np.sqrt(len(comp)))), cmap='bwr')
    axs[c].axis('off')  # Remove axis labels for cleaner visualization
    axs[c].set_title(f'PC {c+1}')


# Place colorbar on the right side vertically
cbar_ax = fig.add_axes([0.92, 0.255, 0.02, 0.47])

fig.colorbar(im, cax=cbar_ax, orientation='vertical', cmap='bwr')
plt.show()

In [ ]:

time = np.linspace(1, 2400, len(correlation_matrices_temp))

colors = ['red', 'blue', 'green', 'orange', 'black', 'purple']

plt.figure(figsize=(16, 8))  # Increase the figure size

for a in range(num_clusters):
    plt.plot(time, distances_temp[a, :], label=f'State {str(a)}', c=colors[a])

plt.xlabel('Time [s]', fontsize=24)
plt.ylabel('1- relative distance', fontsize=24)
plt.legend(fontsize=20)
plt.tick_params(axis='y', labelsize=18)
plt.tick_params(axis='x', labelsize=18)
plt.show()

## Results dif noise, G

### TVB2

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [2]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats2,most_repeating_states2, correlation_matrices2, cluster_labels2,distances2, pca2, reduced_data2 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB5

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [5]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats5,most_repeating_states5, correlation_matrices5, cluster_labels5,distances5, pca5, reduced_data5 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
plot_PCs(pca5)

### TVB7

In [ ]:
path_conn = f'D:/Fariba/15Hz_2/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment55.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats7_2,most_repeating_states7_2, correlation_matrices7_2, cluster_labels7_2,distances7_2, pca7_2, reduced_data7_2 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=950,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods :
        closest_mats7,most_repeating_states7, correlation_matrices7, cluster_labels7,distances7, pca7, reduced_data7 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB8

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [8]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats8,most_repeating_states8, correlation_matrices8, cluster_labels8,distances8, pca8, reduced_data8 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB9

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [9]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats9,most_repeating_states9, correlation_matrices9, cluster_labels9,distances9, pca9, reduced_data9 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Intrahemispheric eeg

In [ ]:
file_path=f'D:/Fariba/5min_surface/'
Subjects=['TVB2','TVB5','TVB7','TVB8','TVB9']#,'TVB10','TVB11','TVB12']
right_indices=[]
left_indices=[]
middle_indices=[]
ordered_ind=[]
for i, subj in enumerate (Subjects) : 
    conn_file=file_path+subj+'/eeg_1010.txt'
    r_ind=[]
    l_ind=[]
    m_ind=[]
    with open(conn_file) as f:
        lines = f.readlines()
    for j, line in enumerate(lines) : 
        words = line.split()
        # Get the first word (element before the first space)
        first_word = words[0]
        print(first_word)
        if first_word[-1].isdigit() : 
            last=int(first_word[-1])
            if last%2==0 : 
                r_ind.append(j)
                print('right')
            elif last%2==1 : 
                l_ind.append(j)
                print('left')
        else :
                m_ind.append(j)
                print('middle')    

    right_indices.append(r_ind)
    left_indices.append(l_ind)
    middle_indices.append(m_ind)
    ordered_ind.append(np.concatenate((l_ind,m_ind,r_ind)))

In [ ]:
ordered_ind=np.concatenate((left_indices[1],middle_indices[1],right_indices[1]))

In [ ]:
ordered_ind

### TVB2

## all subjects eeg

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
closest_mats_joined=[]
for i in [2,5,7,8,9]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(eeg_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)
    closest_mats_joined.append(closest_mats)

In [ ]:

for i, subj in enumerate([2,5,7,8,9]):
    plt.figure()
    plt.imshow(closest_mats_joined[i][0][ordered_ind][ordered_ind],cmap='viridis')
    plt.title(f'Subject {subj}')
    plt.colorbar()
    plt.clim([0,1])
    



In [ ]:
closest_mats_joined[i][0][ordered_ind].shape

In [ ]:
ordered_closest_mats_joined = []
for i,closest_mats in enumerate(closest_mats_joined):
    ordered_closest_mats = []
    for matrix in closest_mats:
        ordered_matrix = matrix[ordered_ind[i]][:, ordered_ind[i]]
        ordered_closest_mats.append(ordered_matrix)
    ordered_closest_mats_joined.append(ordered_closest_mats)

for i, subj in enumerate([2,5,7,8,9]):
    for j in range(3):
        plt.figure()
        plt.imshow(ordered_closest_mats_joined[i][j],cmap='viridis')
        plt.title(f'Subject {subj}')
        plt.colorbar()


In [ ]:
closest_mats_joined = np.array(closest_mats_joined)
closest_mats_joined.shape
for i in closest_mats_joined:
    print(i.shape)
    for j in i: 
        print(j.shape) 
        print(j[ordered_ind][:, ordered_ind].shape)

In [ ]:
ordered_closest_mats_joined = np.array(ordered_closest_mats_joined)
ordered_closest_mats_joined.shape

In [ ]:
ordered_ind[i].shape

# New plan projection Esra

## TVB10

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca10, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False, square=[1,1])

In [ ]:
components = pca10.components_

In [ ]:
new_PC1 = components[0]
new_PC2 = components[1]-components[2]
new_PC3 = components[1]+components[2]
new_PCs=np.array([new_PC1,new_PC2,new_PC3])

for PC in new_PCs : 
    plt.figure()
    plt.imshow(PC.reshape(84,84),cmap='bwr')
    plt.colorbar();plt.clim([-0.05,0.05])

new_PCs=pca10.transform(new_PCs)
new_PC1=new_PCs[0]
new_PC2=new_PCs[1]
new_PC3=new_PCs[2]

In [ ]:
def define_plane(point1, point2):
    # Calculate the normal vector of the plane
    normal = np.cross(point1, point2)
    return normal

# Function to project points onto the plane
def project_onto_plane(plane_normal, plane_point, other_points):
    # Ensure the plane normal is a unit vector
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    # Calculate the equation of the plane (ax + by + cz = d)
    d = np.dot(plane_normal, plane_point)
    
    # Calculate the projection of other points onto the plane
    projections = []
    for point in other_points:
        t = d - np.dot(plane_normal, point)
        projected_point = point + t * plane_normal
        projections.append(projected_point)
    
    return projections

In [ ]:
plan1=define_plane(new_PC1,new_PC2)
plan2=define_plane(new_PC1,new_PC3)
plan3=define_plane(new_PC2,new_PC3)

proj1=np.array(project_onto_plane(plan1, new_PC1, reduced_data))
proj2=np.array(project_onto_plane(plan2, new_PC1, reduced_data))
proj3=np.array(project_onto_plane(plan3, new_PC2, reduced_data))

### PLAN 1

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj1[:-num_clusters, 0], y=proj1[:-num_clusters, 1], z=proj1[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj1[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj1[:-num_clusters, 0],y=proj1[:-num_clusters, 1],z=proj1[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA; axis1=PC1/ axis2=PC2-PC3", 
    height=1000, width=1000)
fig.show()

In [ ]:
def project_3d_to_2d(points_3d, plane_normal):
    # Ensure the plane normal is a unit vector
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    # Calculate the projection of points onto the plane
    # The projection formula: P = V - dot(V, N) * N, where V is the point and N is the normal vector
    projected_points = points_3d - np.dot(points_3d, plane_normal)[:, np.newaxis] * plane_normal
    
    return projected_points

proj1_2d=pd.DataFrame(project_3d_to_2d(proj1,np.cross(proj1[0],proj1[1])))

plt.scatter(proj1_2d[0][:-num_clusters],proj1_2d[1][:-num_clusters],c=cluster_labels)

In [ ]:
colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    proj1_2d[0][:-num_clusters],proj1_2d[1][:-num_clusters], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
fig.update_layout(title=f"Desity plot of projected points", 
    height=1000, width=1000)

# Add crosses marking the projections of the centroids
centroids_proj = project_3d_to_2d(centroids, np.cross(proj1[0], proj1[1]))
fig.add_trace(go.Scatter(x=centroids_proj[:, 0], y=centroids_proj[:, 1],
                         mode='markers+text', marker=dict(symbol='x', size=20,color='white'),text=[str(i) for i in range(num_clusters)],
                         
                         name='Centroids'))



fig.update_layout(title=f"Desity plot of projected points", 
    height=1000, width=1000)
fig.show()

In [ ]:
manual_center=[-0.5,0.5]
xs=proj1_2d[:-num_clusters][0]
ys=proj1_2d[:-num_clusters][1]

dx=manual_center[0]-xs
dy=manual_center[1]-ys

radial_distance=np.sqrt(dx**2+dy**2)
angles=np.arctan2(dy,dx)
unwraped=np.unwrap(angles)
indeg=np.degrees(angles)

In [ ]:
fig=go.Figure(data=[go.Scatter(x=list(range(len(unwraped))),y=unwraped,mode='markers',marker=dict(color=radial_distance,size=5,showscale=True))])
fig.update_layout(coloraxis=dict(colorbar=dict(title='Radial Distance')))
fig.update_layout(title=f"Angle evolution, Subj10, Healthy", 
    height=1000, width=1750)
fig.show()

### PLAN 2

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj2[:-num_clusters, 0], y=proj2[:-num_clusters, 1], z=proj2[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj2[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj2[:-num_clusters, 0],y=proj2[:-num_clusters, 1],z=proj2[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()


In [ ]:
proj2_2d=pd.DataFrame(project_3d_to_2d(proj2,np.cross(proj2[0],proj2[4])))

plt.scatter(proj2_2d[0][:-num_clusters],proj2_2d[1][:-num_clusters],c=cluster_labels)

### PLAN 3

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj3[:-num_clusters, 0], y=proj3[:-num_clusters, 1], z=proj3[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj3[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj3[:-num_clusters, 0],y=proj3[:-num_clusters, 1],z=proj3[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()


In [ ]:
plt.scatter(proj3[:-num_clusters, 2],proj3[:-num_clusters, 0])

In [ ]:
colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    proj3[:-num_clusters, 1], proj3[:-num_clusters, 2], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
plt.scatter()
fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

In [ ]:
con1=proj3[:, 1]/proj3[:, 0]
con2=proj3[:, 2]/proj3[:, 0]
con=np.array([con1,con2])

In [ ]:
plt.scatter(con[0,:-num_clusters],con[1,:-num_clusters],c=cluster_labels)

## Template

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

In [ ]:
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,len(correlation_matrices[0])**2)
reduced_data_2d = PCA(n_components=2).fit_transform(data)
manual_center=[6,0]

colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    reduced_data_2d[:-num_clusters, 0], reduced_data_2d[:-num_clusters, 1], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
fig.add_trace(go.Scatter(x=[manual_center[0]],y=[manual_center[1]],mode='markers',marker=dict(symbol='x',color='black',size=10)))

fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

### Angle and radial distance evolution

In [ ]:
manual_center=[6,0]
xs=reduced_data_2d[:-num_clusters, 0]
ys=reduced_data_2d[:-num_clusters,1]

dx=manual_center[0]-xs
dy=manual_center[1]-ys

radial_distance=np.sqrt(dx**2+dy**2)
angles=np.arctan2(dy,dx)
unwraped=np.unwrap(angles)
indeg=np.degrees(angles)

In [ ]:
plt.plot(indeg)
plt.xlabel('time')
plt.ylabel('angle')
plt.title('Angle of each point with respect to the ellipse center')
plt.figure()
plt.plot(radial_distance)
plt.xlabel('time')
plt.ylabel('radial distance')
plt.title('Radial distance of each point accross time')

In [ ]:
plt.plot(unwraped)
plt.ylabel('unwraped angle')

In [ ]:
fig=go.Figure(data=[go.Scatter(x=list(range(len(unwraped))),y=unwraped,mode='markers',marker=dict(color=radial_distance,size=5,showscale=True))])
fig.update_layout(coloraxis=dict(colorbar=dict(title='Radial Distance')))
fig.update_layout(title=f"Angle evolution", 
    height=1000, width=1750)
fig.show()

In [ ]:
plt.plot(radial_distance,indeg)
plt.title('redial distance vs angle')

### Corr corr Matrix + dendrogram

In [ ]:
corr_corr_mat=np.corrcoef(np.array(correlation_matrices).reshape(len(correlation_matrices),len(correlation_matrices[0])**2))

In [ ]:
plt.imshow(corr_corr_mat)
plt.colorbar()

In [ ]:
linkage_data = linkage(corr_corr_mat, method='ward', metric='euclidean')


In [ ]:
plt.figure(figsize=(25, 10))
k=8
dend=dendrogram(linkage_data, orientation='top', color_threshold=8)  
clusters=scipy.cluster.hierarchy.fcluster(linkage_data, 8, criterion='maxclust')

plt.show()


In [ ]:
plt.figure(figsize=(25, 10))
dend=dendrogram(linkage_data, orientation='top', truncate_mode='lastp', p=8, show_leaf_counts=True, show_contracted=True, no_labels=True, color_threshold=100)  
plt.show()

In [ ]:
fig=ff.create_dendrogram(corr_corr_mat, color_threshold=4.5)
fig.update_layout(width=800, height=500)
fig.show()

In [ ]:
testi=scipy.cluster.hierarchy.fcluster(linkage_data, 8, criterion='maxclust')

In [ ]:
clust1=[]
clust2=[]
clust3=[]
clust4=[]
clust5=[]
clust6=[]
clust7=[]
clust8=[]

clusts=[ [] for _ in range(8) ]

for i in range(len(correlation_matrices)):
    clusts[clusters[i]-1].append(correlation_matrices[i])

for j in range(len(clusts)):
    plt.figure()
    plt.imshow(np.mean(clusts[j],axis=0))

In [ ]:
reorg_matrices = [clusts[0]+clusts[1]+clusts[2]+clusts[3]+clusts[4]+clusts[5]+clusts[6]+clusts[7]] 

In [ ]:
clust1=np.zeros(len(testi[testi==1]))
clust2=np.zeros(len(testi[testi==2]))
clust3=np.zeros(len(testi[testi==3]))
clust4=np.zeros(len(testi[testi==4]))
clust5=np.zeros(len(testi[testi==5]))
clust6=np.zeros(len(testi[testi==6]))
clust7=np.zeros(len(testi[testi==7]))
clust8=np.zeros(len(testi[testi==8]))

for i in range(l)


In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

import numpy as np
from scipy.spatial.distance import pdist, squareform


# get data
data = np.genfromtxt("http://files.figshare.com/2133304/ExpRawData_E_TABM_84_A_AFFY_44.tab",
                     names=True,usecols=tuple(range(1,30)),dtype=float, delimiter="\t")
data_array = data.view((float, len(data.dtype.names)))
data_array = corr_corr_mat


# Initialize figure by creating upper dendrogram
fig = ff.create_dendrogram(data_array, orientation='bottom')
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(data_array, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = pdist(data_array)
heat_data = squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':2000, 'height':2000,
                         'showlegend':False, 'hovermode': 'closest',
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'ticks':""})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Plot!
fig.show()

# Timesteps investigation

## Timestep 0.5

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 600000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

## Timestep 0.3

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.3_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.3),overlap=int(5000*0.5/0.3),tmax=t2/1000,animated_visu=False)

## Timestep 0.2

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.2_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.2),overlap=int(5000*0.5/0.2),tmax=t2/1000,animated_visu=False)

## Timestep 0.1

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.1_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.1),overlap=int(5000*0.5/0.1),tmax=t2/1000,animated_visu=False)

## Timestep 0.05

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.05_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.05),overlap=int(5000*0.5/0.05),tmax=t2/1000,animated_visu=False,ts=0.05)

## Timestep 0.01